## 1.Change mode  to test mode 
```
commands add 
--phase-test True

```

## 2.Verify the performance of the model in the test dataset 
Run the following command, you can see the performance of the model in the test set in the ./submission/ directory.

In [ ]:
!python main.py --config="cfgs/fishnet150-32.yaml" --resume ./checkpoints/fishnet150_bs32/_15.pth.tar  --phase-test True --val True --val-save True --gpus 0

In [ ]:
!python main.py --config="cfgs/fishnet150-32.yaml" --resume ./checkpoints/fishnet150_bs32/_51_best.pth.tar --phase-test True --val True --val-save True --gpus 0

In [ ]:
!python main.py --config="cfgs/fishnet150-32.yaml" --resume ./checkpoints/fishnet150_bs32/_26_best.pth.tar --phase-test True --val True --val-save True 

In [ ]:
!python main.py --config="cfgs/FeatherNet54-32.yaml" --resume ./checkpoints/FeatherNet54/_40_best.pth.tar --phase-test True --val True --val-save True

In [ ]:
!python main.py --config="cfgs/FeatherNet54-se-64.yaml" --resume ./checkpoints/FeatherNet54-se/_68_best.pth.tar --phase-test True --val True --val-save True

In [ ]:
!python main.py --config="cfgs/mobilenetv2.yaml" --resume ./checkpoints/mobilenetv2_bs32/_4_best.pth.tar --phase-test True --val True --val-save True

In [ ]:
!python main.py --config="cfgs/mobilenetv2.yaml" --resume ./checkpoints/mobilenetv2_bs32/_5.pth.tar --phase-test True --val True --val-save True

In [ ]:
!python main.py --config="cfgs/mobilenetv2.yaml" --resume ./checkpoints/mobilenetv2_bs32/_6.pth.tar --phase-test True --val True --val-save True

In [ ]:
!python main.py --config="cfgs/MobileLiteNetA-32.yaml" --resume ./checkpoints/mobilelitenetA_bs32/_50_best.pth.tar --phase-test True --val True --val-save True

In [ ]:
!python main.py --config="cfgs/MobileLiteNetB-32.yaml" --resume ./checkpoints/mobilelitenetB_bs32/_47_best.pth.tar --phase-test True --val True --val-save True

# Test with IR 

In [ ]:
!python main.py --config="cfgs/MobileLiteNetB-32.yaml" --resume ./checkpoints/4mobilelitenetB_bs32-ir/_54.pth.tar --phase-test True --val True --val-save True --phase-ir 3

## 3.split the predicted scores from each submission file

In [9]:
def splitscore(file_dir):
    score = []
    Prefix_str = []
    f = open(file_dir)
    for line in f:
        s =line.split()
        score.append(float(s[-1]))
        s = s[0] + ' ' + s[1] + ' ' + s[2] + ' '
        Prefix_str.append(s)
    return score,Prefix_str

## 4.Collect all predicted scores

In [ ]:
from pathlib import Path  
# specify directory that contains the prediction scores of all models
data_dir = '/home/zp/disk1T/libxcam-testset/submission/checkpoints_testset_predictes'
txt_files = [i for i in Path(data_dir).glob("**/*.txt") ] 

file_dir1=txt_files[0]
print('1',txt_files[0])
score1,Prefix_str = splitscore(file_dir1)

file_dir2 = txt_files[1]
print('2',txt_files[1])
score2,Prefix_str = splitscore(file_dir2)

file_dir3 = txt_files[2]
print('3',txt_files[2])
score3,Prefix_str = splitscore(file_dir3)

file_dir4 = txt_files[3]
print('4',txt_files[3])
score4,Prefix_str = splitscore(file_dir4)

file_dir5 = txt_files[4]
print('5',txt_files[4])
score5,Prefix_str = splitscore(file_dir5)

file_dir6 = txt_files[5]
print('6',txt_files[5])
score6,Prefix_str = splitscore(file_dir6)

file_dir7 = txt_files[6]
print('7',txt_files[6])
score7,Prefix_str = splitscore(file_dir7)

file_dir8 = txt_files[7]
print('8',txt_files[7])
score8,Prefix_str = splitscore(file_dir8)

file_dir9 = txt_files[8]
print('9',txt_files[8])
score9,Prefix_str = splitscore(file_dir9)

file_dir10 = txt_files[9]
print('10',txt_files[9])
score10,Prefix_str = splitscore(file_dir10)

scores = [score1,score2,score3,score4,score5,score6,score7,score8,score9,score10]


In [12]:
from sklearn.metrics import confusion_matrix
import roc
import numpy as np
def get_ensembled_scores(threshold_min,threshold_max,ir_threshold, fish_threshold,scores,ir_score,fish_scores,factors):
    ensembled_scores = []
    for i in range(len(ir_score)):
        line_socres = [scores[j][i] for j in range(10)]
        mean_socre = np.array(line_socres).dot(np.array(factors)) / np.sum(np.array(factors))
        if mean_socre < threshold_min:
            ensembled_scores.append(min(line_socres))
        elif (mean_socre > threshold_max):
            ensembled_scores.append(max(line_socres))
        else:
            if fish_scores[i] < fish_threshold:
                ensembled_scores.append(min(ir_score[i],min(line_socres),fish_scores[i]))
            elif ir_score[i] < ir_threshold:
                ensembled_scores.append(min(ir_score[i],min(line_socres),fish_scores[i]))
            else:
                if (np.sum(np.array(factors)) * mean_socre + fish_scores[i])/(np.sum(np.array(factors))+1) > 0.5:
                    ensembled_scores.append(max(fish_scores[i],max(line_socres)))
                else:
                    ensembled_scores.append(min(ir_score[i],min(line_socres)))
    return ensembled_scores

def get_prediceted_labels(ensembled_score):
    prediceted_label = []
    for i in range(len(ensembled_score)):
        if ensembled_score[i] > 0.5:
            prediceted_label.append(1)
        else:
            prediceted_label.append(0)
    return prediceted_label

In [16]:
ir_predict_file = 'submission/IRpredicte57710_mobilelitenetB_55_submission.txt' # 对所有的IR图片进行一次前向预测得到的结果
ir_scores,ir_Prefix_str = splitscore(ir_predict_file)


fish150_51 = 'submission/checkpoints_testset_predictes/2019-03-08_19:01:11_fishnet150_52_submission.txt' 
fish150_scores,fish150_Prefix_str = splitscore(fish150_51)

factors = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
threshold_min = 0.18
threshold_max = 0.8
ir_threshold = 0.54
fish_threshold = 0.32
ensembled_scores = get_ensembled_scores(threshold_min,threshold_max,ir_threshold, fish_threshold,scores,ir_scores,fish150_scores,factors)
prediceted_labels = get_prediceted_labels(ensembled_scores)

#  save submission file 

In [25]:
test_file_list = open('data/test_public_list.txt').readlines()
submission_file = open('submission/submission.txt','w')
for i in range(len(ensembled_scores)):
    submission_file.write(test_file_list[i].rstrip('\n') + str(ensembled_scores[i]) + '\n')
submission_file.close()